In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_1 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_account_x_balance.csv")
data_2 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_party_products.csv")
data_3 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_party_x_socdem.csv")
data_4 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_story_logs.csv")
data_5 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_story_texts.csv")
data_6 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_test_transactions.csv")
data_7 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_train_transactions.csv")

In [ ]:
data_7['transaction_dttm'] = pd.to_datetime(data_7['transaction_dttm'])

In [ ]:
unique_dates = data_7['transaction_dttm'].map(lambda dt: dt.replace(day=1))

In [ ]:

u_dat = pd.DataFrame(unique_dates.unique(), columns=['transaction_dttm'])
u_cat= pd.DataFrame(data_7.category.unique(), columns=['category'] )
u_id = pd.DataFrame(data_7.party_rk.unique(), columns=['party_rk'] )

u_dat['key'] = 1
u_cat['key'] = 1
u_id['key'] = 1

u_data7 = pd.merge(u_id, 
                  pd.merge(u_dat, u_cat, on ='key')).drop("key", 1)

# u_data7 = u_data7.drop([0,'0_x','0_y'],1)

In [ ]:
u_data7

,party_rk,transaction_dttm,category
0,20337,2019-01-01,Сувениры
1,20337,2019-01-01,Фаст Фуд
2,20337,2019-01-01,Супермаркеты
3,20337,2019-01-01,Дом/Ремонт
4,20337,2019-01-01,Сервисные услуги
...,...,...,...
22199995,92718,2019-12-01,Duty Free
22199996,92718,2019-12-01,Турагентства
22199997,92718,2019-12-01,Образование
22199998,92718,2019-12-01,Искусство


In [ ]:
df_main = data_7[['transaction_dttm', 'party_rk', 'category', 'transaction_amt_rur']]
df_main.transaction_amt_rur = df_main['transaction_amt_rur'].fillna(-11092021.0)

df_main2 = df_main.merge(u_data7, on=['transaction_dttm', 'party_rk', 'category'], how = 'right')
df_main2.transaction_amt_rur = df_main2['transaction_amt_rur'].fillna(0)
df_main2.transaction_amt_rur = df_main2.transaction_amt_rur.apply(lambda x: np.nan if x ==-11092021.0 else x)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df_main_sort = df_main2.sort_values(by=['transaction_dttm','party_rk'])

In [ ]:
df_main_sort.to_csv("df_main_sort.csv", index=False)